In [ ]:
import requests  # 发送请求
from bs4 import BeautifulSoup  # 解析网页
import pandas as pd  # 存取csv
from time import sleep  # 等待时间
import re  # 正则表达式
from requests.exceptions import RequestException  # 异常处理

In [ ]:
#爬取 基本信息 剧情简介 演职员表
movie_name = []  # 电影名称
movie_url = []  # 电影链接
movie_star = []  # 电影评分
movie_star_people = []  # 评分人数

movie_director = []  # 导演
movie_screenwriter = []  # 编剧
movie_actor = []  # 主演

movie_year = []  # 上映年份
movie_country = []  # 国家
movie_type = []  # 类型

movie_date = []  # 上映日期
movie_last_raw = []  # 片长原始数据
movie_last = []  # 片长
movie_language = []  # 语言
movie_bias = []  # 别名
movie_all_name = []  #名称+别名
movie_intro = []  # 剧情简介
movie_each_last_raw = [] #单集片长原始数据
movie_each_last = [] #单集片长
movie_chapnum = [] #集数
movie_season = [] #季数

In [ ]:
def get_movie_info(url, headers):
    """
    获取豆瓣电影详情数据
    :param url: 爬取地址
    :param headers: 爬取请求头
    :return: None
    """
    #检验推送
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    """
	soup.select(‘标签名'),代表根据标签来筛选出指定标签。
    CSS中#xxx代表筛选id，soup.select(‘#xxx')代表根据id筛选出指定标签,返回值是一个列表。
    CSS中.###代表筛选class，soup.select('.xxx')代表根据class筛选出指定标签,返回值是一个列表。
    嵌套select: soup.select(“#xxx .xxxx”)，如(“#id2 .news”)就是id=”id2”标签下class=”news的标签，返回值是一个列表。
	"""
    # print(soup.prettify())
    # 电影名称
    name_existed = soup.select('#content h1 span[property="v:itemreviewed"]')
    if name_existed:
        name = name_existed[0].text.strip()
        print(name)
        movie_name.append(name) # append 用于在列表末尾添加新的对象
    else:
        movie_name.append("")
    '''name=soup.select('#content h1 span[property="v:itemreviewed"]')[0].text.strip()
    print(name)
    movie_name.append(name)''' # append 用于在列表末尾添加新的对象
    # 电影年份
    year_existed = soup.select('#content h1 span.year') 
    if year_existed:
        year = year_existed[0].text.strip()
        year = year.strip('()')
        print(year)
        movie_year.append(year)
    else:
        movie_year.append("")
    '''year=soup.select('#content h1 span.year')[0].text.strip()
    year=year.strip('()')
    print(year)
    movie_year.append(year)'''
    # 电影评分
    if soup.select('.rating_num'):
        star = soup.select('.rating_num')[0].text.strip()  # 电影评分
        print(star)
        movie_star.append(star)
    else:
        movie_star.append("")
    # 评分人数
    if soup.select('.rating_sum a span[property="v:votes"]'):
        star_people = soup.select('.rating_sum a span[property="v:votes"]')[0].text  # 评分人数
        print(star_people)
        movie_star_people.append(star_people)
    else:
        movie_star_people.append("")
    # info
    info = soup.select('#info')
    if info:
        info_2_text = info[0].text
        #print(info_2_text)
        #print(info)
        # 导演
        dir_len=len(info[0].select('a[rel="v:directedBy"]'))
        director=''
        for i in range(dir_len):
            director = director + info[0].select('a[rel="v:directedBy"]')[i].text.strip() + ' / '
        director=director.strip(' / ')
        print(director)
        movie_director.append(director)
        # 编剧 html没有ref
        """
        scr_len=len(info[0].select('span[class="attrs"]'))
        print(scr_len)
        screenwriter=''
        i=0
        for i in range(scr_len):
            screenwriter = screenwriter + info[0].select('span[class="attrs"]')[i].text.strip() + ' / '
        print(screenwriter)
        movie_screenwriter.append(screenwriter)
        """
        # 主演
        act_len=len(info[0].select('a[rel="v:starring"]'))
        actor=''
        for i in range(act_len):
            actor = actor + info[0].select('a[rel="v:starring"]')[i].text.strip() + ' / '
        actor=actor.strip(' / ')
        print(actor)
        movie_actor.append(actor)
        # 上映日期
        date_len=len(info[0].select('span[property="v:initialReleaseDate"]'))
        date=''
        for i in range(date_len):
            date = date + info[0].select('span[property="v:initialReleaseDate"]')[i].text.strip() + ' / '
        date=date.strip(' / ')
        print(date)
        movie_date.append(date)
        # 类型
        type_len=len(info[0].select('span[property="v:genre"]'))
        type=''
        for i in range(type_len):
            type = type + info[0].select('span[property="v:genre"]')[i].text.strip() + ' / '
        type=type.strip(' / ')
        print(type)
        movie_type.append(type)
    else:
        info_2_text = ''
        movie_director.append("")
        movie_actor.append("")
        movie_date.append("")
        movie_type.append("")
        
    #季数
    try:
        season_text = soup.select('#info #season')[0].get_text()
        total_len = len(season_text)
        print(total_len)
        if total_len>9:
            season=season_text[-2]+season_text[-1]
        else:
            season=season_text[-1]
    except:
        season=''
    print('season={}'.format(season))
    movie_season.append(season)


    #处理没有ref的
    #直接按\n切有问题
    #先切掉IMDb及之后的信息
    split_info=info_2_text.split('\nIMDb:')[0]
    print(split_info)
    #找别名 有的没有别名
    try:
        bias=split_info.split('又名:')[1].split('\n')[0].strip()
    except:
        bias=''
    print(bias)
    movie_bias.append(bias)
    split_info=split_info.split('\n又名:')[0] #切掉别名
    # 片长
    try:
        last_raw=soup.select('#info span[property="v:runtime"]')[0].text.strip()
        
    except:
        try:
            last_raw=split_info.split('\n片长:')[1].split('\n')[0].strip()
        except:
            last_raw=''
    last=last_raw.split('分')[0].split(':')[-1].strip()
    print(last_raw) 
    print(last)
    movie_last_raw.append(last_raw)
    movie_last.append(last)
    #切掉片长
    split_info=split_info.split('\n片长:')[0]
    #找单集片长
    try:
        each_last_raw=split_info.split('单集片长:')[1].split('\n')[0].strip()
    except:
        each_last_raw=''
    each_last=each_last_raw.split('分')[0].split(':')[-1].strip()
    print(each_last)
    movie_each_last_raw.append(each_last_raw)
    movie_each_last.append(each_last)
    split_info=split_info.split('\n单集片长:')[0] #切掉单集片长
    #找集数
    try:
        chapnum=split_info.split('集数:')[1].split('\n')[0].strip()
    except:
        chapnum=''
    print(chapnum)
    movie_chapnum.append(chapnum)
    split_info=split_info.split('\n集数:')[0] #切掉集数
    #切掉上映日期
    split_info=split_info.split('\n上映日期:')[0]
    #找语言
    try:
        language = split_info.split('语言:')[1].split('\n')[0].strip()
    except:
        language=''
    print(language)
    movie_language.append(language)
    split_info=split_info.split('\n语言:')[0] #切掉语言
    #找国家地区
    try:
        country = split_info.split('制片国家/地区:')[1].split('\n')[0].strip()
    except:
        country=''
    print(country)
    movie_country.append(country)
    split_info=split_info.split('\n制片国家/地区:')[0] #切掉国家地区
    #o_info_num=len(split_info)
    #print(o_info_num)
    #编剧
    try:
        screenwriter = split_info.split('\n')[2].split('编剧:')[1].strip()
    except:
        screenwriter = ''
    print(screenwriter)
    movie_screenwriter.append(screenwriter)
    # 名称+别名
    if name_existed:
        name = name_existed[0].text.strip()
        if(len(bias)>0):
            all_name = name + ' / ' + bias
        else:
            all_name = name
        print(all_name)
        movie_all_name.append(all_name)
    else:
        movie_all_name.append("")
    #剧情简介
    try:
        intro=soup.select('.related-info .indent span[class="all hidden"]')[0].text.replace('\n','').strip() #删去换行
    except: #没有展开
        intro_existed = soup.select('.related-info .indent span[property="v:summary"]')
        if intro_existed:
            intro = intro_existed[0].text.replace('\n', '').strip()
        else:
            intro = ''
        #intro=soup.select('.related-info .indent span[property="v:summary"]')[0].text.replace('\n','').strip()
    intro = re.sub(r"\s+", " ", intro) #连续多个空格只保留一个    
    print(intro)
    movie_intro.append(intro)


    # for movie in soup.select('.item'):
        # print(movie.select('.hd1'))
        # name = movie.select('.h1')[0].text.replace('\n', '')  # 电影名称
        # print(len(movie.select('.hd a'))) # 列表长度为1
        #选择class为hd的元素下所有的a标签，并返回一个列表
        #.text获取元素的文本内容
        #.replace('\n', '')将换行符替换为空，即去掉电影名中的换行符
        # 
        # movie_name.append(name) # append 用于在列表末尾添加新的对象
"""
        url = movie.select('.hd a')[0]['href']  # 电影链接
        #'href' html中的超链接属性
        movie_url.append(url)

        
        star_people = movie.select('.star span')[3].text  # 评分人数
        star_people = star_people.strip().replace('人评价', '')
        movie_star_people.append(star_people)
        
        movie_infos = movie.select('.bd p')[0].text.strip()  # 导演、主演、年份、国家、类型
        #strip 去掉字符串首尾的空格
        
        director = movie_infos.split('\n')[0].split('   ')[0]
        movie_director.append(director)
        try:  # 页面上既有导演，又有主演
            actor = movie_infos.split('\n')[0].split('   ')[1]
            movie_actor.append(actor)
        except:  # 页面上只有导演，没有主演
            movie_actor.append(None)
        if name == '大闹天宫 / 大闹天宫 上下集  /  The Monkey King':  # 大闹天宫，特殊处理
            year0 = movie_infos.split('\n')[1].split('/')[0].strip()
            year1 = movie_infos.split('\n')[1].split('/')[1].strip()
            year2 = movie_infos.split('\n')[1].split('/')[2].strip()
            year = year0 + '/' + year1 + '/' + year2
            movie_year.append(year)
            country = movie_infos.split('\n')[1].split('/')[3].strip()
            movie_country.append(country)
            type = movie_infos.split('\n')[1].split('/')[4].strip()
            movie_type.append(type)
        else:  # 其他电影，正常处理
            year = movie_infos.split('\n')[1].split('/')[0].strip()
            movie_year.append(year)
            country = movie_infos.split('\n')[1].split('/')[1].strip()
            movie_country.append(country)
            type = movie_infos.split('\n')[1].split('/')[2].strip()
            movie_type.append(type)
"""


In [ ]:
def save_to_csv(csv_name):
    """
    数据保存到csv
    :return: None
    """
    df = pd.DataFrame()  # 初始化一个DataFrame对象，二维表格
    df['url'] = movie_url
    df['电影名称'] = movie_name
    df['别名'] = movie_bias
    df['全部名称'] = movie_all_name
    df['电影评分'] = movie_star
    df['评分人数'] = movie_star_people
    df['导演'] = movie_director
    df['编剧'] = movie_screenwriter
    df['主演'] = movie_actor
    df['上映年份'] = movie_year
    df['首映日期'] = movie_date
    df['国家'] = movie_country
    df['语言'] = movie_language
    df['片长raw'] = movie_last_raw
    df['片长'] = movie_last
    df['季数'] = movie_season
    df['集数'] = movie_chapnum
    df['单集片长raw'] = movie_each_last_raw
    df['单集片长'] = movie_each_last
    df['类型'] = movie_type
    df['剧情简介'] = movie_intro
    df.to_csv(csv_name, encoding='utf_8_sig')  # 将数据保存到csv文件

In [ ]:

if __name__ == "__main__":
    # 定义一个请求头(防止反爬)
    headers = {
        #'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36(KHTML, like Gecko)Chrome/118.0.0.0 Safari/537.36 Edg/118.0.2088.76'}
    # 开始爬取豆瓣数据
    data = pd.read_csv(r'./id/Movie_id.csv',sep=',',header=None,usecols=[0])
    array=data.values[0::,0]   #读取全部行，第一列
    print(array[0])              #array是数组形式存储，顺序与data读取的数据顺序格式相同
    num=len(array)
    print(num)
    part= 0
    group=100
    for j in range(part*group,(part+1)*group):#(324,325):
        #id=1292052
        id_test=1
        try:
            id=array[j]
        except:
            break
        page_url = 'https://movie.douban.com/subject/{}/'.format(id)
        movie_url.append(page_url)  #
        #{}是占位符 .format()是格式化函数，括号内内容替换占位符
        print('开始爬取第{}个，id={}，地址是:{}'.format(j, id, page_url))
        #try:
        get_movie_info(page_url, headers)
        #sleep(1)  # 等待1秒(防止反爬)
        #except: #出错
        #    print('请检查url')
        # 保存到csv文件
    save_to_csv(csv_name="./movie/MovieDouban_test.csv".format(part)) #.csv"
